## Import Statements

In [1]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

from python_speech_features import mfcc as mfccc
from python_speech_features import logfbank
import scipy.io.wavfile as wav
from pandas import DataFrame

ImportError: No module named 'librosa'

## Data Preprocessing

In [3]:
header = ''
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [ ]:
file = open('data1.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)

### Preparation of dataset

In [10]:
genres = 'NS S'.split()
for g in genres:
    for filename in os.listdir(f'./{g}'):
        songname = f'./{g}/{filename}'
        (rate,sig) = wav.read(songname)
        mfcc = mfccc(sig,rate,numcep=20)    
        for e in mfcc:
            to_append = f''
            for z in e:
                to_append += f' {z}'
            to_append += f' {g}'
            file = open('data2.csv', 'a', newline='')
            with file:
                writer = csv.writer(file)
                writer.writerow(to_append.split())
            file.close()

## Data

In [52]:
d = pd.read_csv('./data/data1.csv')
data = DataFrame.drop_duplicates(d)
data.tail()

,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
1173610,12.932554,-3.438311,0.712146,1.496589,6.448451,2.992739,-0.824358,7.651176,-15.793766,7.669075,...,5.759200,-7.525948,8.672894,-3.928161,-13.454528,0.736362,-5.667800,1.932495,1.284689,NS
1173611,13.078408,-2.660954,1.599874,-2.059938,10.321796,6.187514,4.866676,10.455900,-20.453577,6.245536,...,3.035482,-8.298606,0.402892,-7.453174,-11.439127,0.448788,-3.902659,2.115690,1.639411,NS
1173612,12.958079,-1.267645,-0.654874,-2.410546,8.801666,1.258751,0.438309,14.905351,-29.494555,3.738767,...,4.619153,-2.773699,2.339311,-4.887733,-7.183047,2.242666,-5.891264,1.602447,-0.388524,NS
1173613,13.067278,-2.336895,1.597430,-0.662740,8.254957,-2.752948,2.668889,14.964874,-25.839926,6.372458,...,3.512694,-6.212925,-1.443155,-5.001671,-6.875766,1.695962,-5.819479,3.336239,-1.606121,NS
1173614,13.046771,-2.683565,-0.045187,3.630588,11.426692,0.959028,6.029484,7.630195,-23.338402,13.703065,...,3.611029,-11.770231,-6.604062,-10.325389,-11.760047,5.464849,-4.793835,4.531965,-1.609620,NS


### Preprocessing

In [56]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [57]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Model

In [59]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(256, activation='relu'))

model.add(layers.Dense(512, activation='relu'))

model.add(layers.Dense(256, activation='relu'))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(2, activation='softmax'))

In [60]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [61]:
history = model.fit(X_train,y_train,epochs=5,batch_size=64)

Epoch 1/5
821530/821530 [==============================] - 124s 151us/step - loss: 0.2816 - acc: 0.8813
Epoch 2/5
821530/821530 [==============================] - 125s 153us/step - loss: 0.2650 - acc: 0.8884
Epoch 3/5
821530/821530 [==============================] - 121s 147us/step - loss: 0.2596 - acc: 0.8903
Epoch 4/5
821530/821530 [==============================] - 121s 148us/step - loss: 0.2564 - acc: 0.8917s - loss: 0.
Epoch 5/5
821530/821530 [==============================] - 123s 150us/step - loss: 0.2537 - acc: 0.8924


## Evaluvate the model

In [66]:
test_loss, test_acc = model.evaluate(X_test,y_test)

print('test_acc: ',test_acc)

352085/352085 [==============================] - 20s 57us/step
test_acc:  0.8912506923052615


## Save Model

In [63]:
# save model and architecture to single file
model.save("model_128_128.h5")
print("Saved model to disk")

Saved model to disk


## Audio Classification and Segmentation

In [1]:
from pydub import AudioSegment
from keras.models import load_model
from python_speech_features import mfcc as mfccc
import scipy.io.wavfile as wav

def predict(f):
    
    model = load_model('model_final.h5')
    
    (rate,sig) = wav.read(f)
    mfcc = mfccc(sig,rate,numcep=20)
    pred = model.predict_classes(mfcc)

    newAudio = AudioSegment.from_wav(f)
    
    a=[]
    time = 0
    starttime = 0
    pointer = pred[0]
    prev = pred[0]
    for i in range(len(pred)-1):
        if(pointer!=prev):
            if(pointer==1):
                if(pred[i+1]==0):
                    pointer = pred[i+1]
                    time = time + 10
                    continue
                else:
                    starttime = time

            elif (pointer == 0):
                if(pred[i+1]==1):
                    pointer = pred[i+1]
                    time = time + 10
                    continue
                else:
                    a.append([starttime,time-10])
                print('Segment at {}ms'.format(time-10))
            prev=pointer

        pointer = pred[i+1]
        time = time + 10

    if(prev==1 and pred[-1]==1):
        a.append([starttime,time-10])
        print('Segment at {}ms'.format(time-10))
        
    for i,j in enumerate(a):
    
        t1 = j[0]
        t2 = j[1]

        audio = newAudio[t1:t2]
        audio.export('./'+str(i)+f[2:], format="wav")
        
    return pred

C:\Users\Sudharshan\Anaconda3\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
Using TensorFlow backend.


In [2]:
pred = predict('./katl_dep_0927_20170105_sr_001.wav')

W0802 08:36:56.038360  1940 deprecation_wrapper.py:119] From C:\Users\Sudharshan\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0802 08:36:56.067392  1940 deprecation_wrapper.py:119] From C:\Users\Sudharshan\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0802 08:36:56.193533  1940 deprecation_wrapper.py:119] From C:\Users\Sudharshan\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0802 08:36:56.198517  1940 deprecation_wrapper.py:119] From C:\Users\Sudharshan\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0802 08:36:56.199727  1940 deprecation_wrapper

Segment at 18890ms
Segment at 27440ms
Segment at 33250ms
Segment at 34560ms
Segment at 34850ms
Segment at 35020ms
Segment at 35690ms
Segment at 35940ms
Segment at 36270ms
Segment at 37360ms
Segment at 37430ms
Segment at 38060ms
Segment at 39060ms
Segment at 68150ms
Segment at 72320ms
Segment at 80040ms
Segment at 94570ms
Segment at 94690ms
Segment at 96020ms
Segment at 100350ms
Segment at 107420ms
Segment at 123000ms
Segment at 123050ms
Segment at 153630ms
Segment at 153680ms
Segment at 153770ms
Segment at 153880ms
Segment at 154050ms
Segment at 160480ms
Segment at 163200ms
Segment at 166670ms
Segment at 167340ms
Segment at 167500ms
Segment at 229540ms
Segment at 265170ms
Segment at 266690ms
Segment at 268350ms
Segment at 276210ms
Segment at 276890ms
Segment at 303250ms
Segment at 309280ms
Segment at 320910ms
Segment at 386130ms
Segment at 386260ms
Segment at 386400ms
Segment at 386440ms
Segment at 386540ms
Segment at 386630ms
Segment at 386700ms
Segment at 399490ms
Segment at 403520ms

In [3]:
print("Prediction:\n",pred)

Prediction:
 [1 1 1 ... 1 1 1]


In [ ]:
!pip install librosa